In [18]:
import pandas as pd
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\With_ProductId.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7777 entries, 0 to 7776
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ProductId              7777 non-null   int64 
 1   SterilizationMethodId  7777 non-null   int64 
 2   MachineTypeId          7777 non-null   int64 
 3   ItemTypeId             7777 non-null   int64 
 4   SpecialtyId            7777 non-null   int64 
 5   CategoryName           7777 non-null   object
dtypes: int64(5), object(1)
memory usage: 364.7+ KB


In [4]:
# Group by CategoryName and count occurrences
category_counts = df['CategoryName'].value_counts()
category_counts

CategoryName
Instrument        5501
Power System       943
Rigid Scope        845
Flexible Scope     274
Video              103
Ultrasound          74
Ophthalmic          24
Endocam             13
Name: count, dtype: int64

In [8]:
# Downsample Instrument category to 2000 instances
instrument_df = df[df['CategoryName'] == 'Instrument']
downsampled_instrument_df = resample(instrument_df, replace=False, n_samples=1800, random_state=42)

# Combine downsampled "Instrument" with other categories
final_df = pd.concat([downsampled_instrument_df, df[df['CategoryName'] != 'Instrument']])

In [13]:
# Group by CategoryName and count occurrences
category_counts = final_df['CategoryName'].value_counts()
category_counts

CategoryName
Instrument        1800
Power System       943
Rigid Scope        845
Flexible Scope     274
Video              103
Ultrasound          74
Ophthalmic          24
Endocam             13
Name: count, dtype: int64

In [14]:

# Group categories into 'Others'
final_df.loc[final_df['CategoryName'].isin(['Flexible Scope', 'Video', 'Ultrasound', 'Ophthalmic', 'Endocam']), 'CategoryName'] = 'Others'

# Separate categories 'Instrument', 'Power System', and 'Rigid Scope'
instrument_df = final_df[final_df['CategoryName'] == 'Instrument']
power_system_df = final_df[final_df['CategoryName'] == 'Power System']
rigid_scope_df = final_df[final_df['CategoryName'] == 'Rigid Scope']

# Concatenate all DataFrames
new_df = pd.concat([instrument_df, power_system_df, rigid_scope_df, final_df[final_df['CategoryName'] == 'Others']])

# Display the updated counts
print(new_df['CategoryName'].value_counts())


CategoryName
Instrument      1800
Power System     943
Rigid Scope      845
Others           488
Name: count, dtype: int64


In [22]:
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\New_categories.xlsx')

In [23]:
print(df['CategoryName'].value_counts())

CategoryName
Instrument      1499
Power System     800
Rigid Scope      750
Others           434
Name: count, dtype: int64


In [24]:
# Combine categories except "Instrument" into one category called "Others"
def combine_categories(df):
    df.loc[~df['CategoryName'].isin(['Instrument']), 'CategoryName'] = 'Other'
    return df

def segregate(df):
    X = df.drop(columns=['CategoryName'])
    y = df['CategoryName']
    return X, y
    
# Train and evaluate RandomForestClassifier
def train_and_evaluate_model(X, y):
    model = RandomForestClassifier(random_state=42)
    
    model.fit(X, y)
    
    # Predict on the training and testing data
    predictions = model.predict(X)
    
    return model

# Preprocess the data
df1 = df.copy()
df2 = df.copy()

# Model 1: Classify Instrument vs. Others
df1 = combine_categories(df1)

X1, y1 = segregate(df1)

print("Model 1:")

model_first = train_and_evaluate_model(X1, y1)

test_df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\TestCases_PId.xlsx')

# Iterate over each row in the test data
for index, row in test_df.iterrows():
    # Extract the input features from the row
    new_input = row[['SterilizationMethodId', 'MachineTypeId', 'ItemTypeId', 'SpecialtyId']].values.reshape(1, -1)
    # Make predictions using the trained models
    predicted_category = model_first.predict(new_input)

    if predicted_category == 'Other':
        # Model 4: Classify 'Ultrasound', 'Opthamlmic', 'Endocam'
        df2 = df2[df2['CategoryName'].isin(['Power System', 'Rigid Scope', 'Others'])]

        X2, y2 = segregate(df2)
               
        print("\nModel 2:")
        model_last = train_and_evaluate_model(X2, y2)
        
        predicted_category = model_last.predict(new_input)
        print("Final Predicted Category from Model 2:", predicted_category)
        # Update the 'Predicted_Values' column with the predicted category
        test_df.at[index, 'Predicted_Values'] = predicted_category

    else:
        print("Final Predicted Category from Model 1:", predicted_category)
        # Update the 'Predicted_Values' column with the predicted category
        test_df.at[index, 'Predicted_Values'] = predicted_category
        
# Save the updated DataFrame back to the Excel file
test_df.to_excel(r'C:\Users\pawvaibh\Downloads\Predicted_Output_2.xlsx', index=False)

Model 1:


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\Temp\ipykernel_14112\328415599.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Instrument' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_df.at[index, 'Predicted_Values'] = predicted_category
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Final Predicted Category from Model 1: ['Instrument']

Model 2:


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Final Predicted Category from Model 2: ['Power System']

Model 2:


...

C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Final Predicted Category from Model 2: ['Others']

Model 2:
Final Predicted Category from Model 2: ['Rigid Scope']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



Model 2:
Final Predicted Category from Model 2: ['Rigid Scope']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [25]:

# Read the predicted values and true values from the Excel file
val_df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\Predicted_Output_2.xlsx')

# Extract true and predicted values
true_values = val_df['True_Values']
predicted_values = val_df['Predicted_Values']

# Get unique classes from both true and predicted values
unique_classes_true = true_values.unique()
unique_classes_predicted = predicted_values.unique()

# Calculate evaluation metrics
accuracy = accuracy_score(true_values, predicted_values)
precision = precision_score(true_values, predicted_values, average='weighted')
recall = recall_score(true_values, predicted_values, average='weighted')
f1 = f1_score(true_values, predicted_values, average='weighted')
conf_matrix = confusion_matrix(true_values, predicted_values)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Print the unique classes
print("Unique classes in true labels:", unique_classes_true)
print("Unique classes in predicted labels:" , unique_classes_predicted)

print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.5295109612141653
Precision: 0.5251338669709907
Recall: 0.5295109612141653
F1 Score: 0.5224657156376518
Unique classes in true labels: ['Instrument' 'Power System' 'Rigid Scope' 'Others']
Unique classes in predicted labels: ['Instrument' 'Power System' 'Rigid Scope' 'Others']
Confusion Matrix:
[[207   6  63  25]
 [ 11  13   5  25]
 [ 64   7  37  35]
 [ 17  12   9  57]]
